Nombre: Jessica Méndez

Materia: PLN, Visión y Ética Computacional

02 de Febrero de 2023


---



---

**Tecmilenio y la creadora del presente notebook, no guarda relación alguna con las marcas mencionadas como ejemplo. Las marcas son propiedad de sus titulares conforme a la legislación aplicable, se utilizan con fines académicos y didácticos, por lo que no existen fines de lucro, relación publicitaria o de patrocinio.

---


# Instalación de Paquetes

In [ ]:
!pip install NLTK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Bibliotecas a usar

In [ ]:
import nltk
import random
from nltk.corpus import names
from nltk.corpus import movie_reviews


 
nltk.download('names')
nltk.download('movie_reviews')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

# Entrenamiento de un clasificador bayesiano

## El proceso de clasificación de textos incluye los siguientes pasos:

**Preparación de los datos:** en este paso se selecciona el conjunto de datos con el que se trabajará y se realiza un preprocesamiento básico al texto para después separarlo en el conjunto de entrenamiento y de prueba.

**Extracción de características**: el texto se transforma en elementos característicos que se usarán en el proceso de clasificación.

**Entrenamiento del modelo:** el clasificador se entrena con los datos etiquetados.

**Evaluación del modelo**: el clasificador se evalúa y, en caso de ser necesario, se realizan ajustes para mejorar el resultado.


El primer ejemplo trata de clasificar nombres de personas según el género. Se utilizará la librería NLTK (Bird, Klein y Loper, 2009) y el corpus names. En la figura 2 se muestra la descarga de los datos y una breve exploración sobre ellos. En total hay 7,944 nombres clasificados como masculinos o femeninos.

In [ ]:
nombres = ([(nombre,'male') for nombre in names.words('male.txt')]+[(nombre,'female') for nombre in names.words('female.txt')])
random.shuffle(nombres)

In [ ]:
print('Total de nombres:', len(nombres), '\nEjemplo de nombre femenino :', nombres[0], '\nEjemplo de nombre masculino', nombres[7000])

Total de nombres: 7944 
Ejemplo de nombre femenino : ('Gloriane', 'female') 
Ejemplo de nombre masculino ('Brunhilde', 'female')


Para la **extracción** de características se ha decidido considerar solo la última letra del nombre, ya que en el idioma inglés los nombres que terminan con la letra k, o, r, s y t son más probables de pertenecer al género masculino y aquellos que terminan con a, e, i al femenino. Se implementó una función para extraer la última letra de cada instancia del conjunto de datos como se observa en la figura 3.

In [ ]:
def caracteristica_genero(word):
  return {'Ultima leta': word[-1]}

In [ ]:
caracteristica_genero('Carlo')

{'Ultima leta': 'o'}

Para entrenar al modelo, separamos el total de los datos en los conjuntos de entrenamiento y de prueba, siguiendo la recomendación del 70% de los datos al conjunto de entrenamiento y 30% al de prueba. Dado que el total de registros es de 7,944, el 70% corresponde aproximadamente a 5,560 y el complemento deberá ser aproximadamente el 30%.

Con el conjunto de entrenamiento se entrena al clasificador y en este caso se utilizará el NaiveBayesClassifier de NLTK. El proceso se describe en la figura 4.

In [ ]:
caract_conjunto = [(caracteristica_genero(n), g) for (n,g) in nombres]

In [ ]:
caract_conjunto[:5]

[({'Ultima leta': 'e'}, 'female'),
 ({'Ultima leta': 'n'}, 'male'),
 ({'Ultima leta': 'l'}, 'male'),
 ({'Ultima leta': 'l'}, 'female'),
 ({'Ultima leta': 'y'}, 'female')]

In [ ]:
train_set, test_set = caract_conjunto[:5560], caract_conjunto[5560:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.classify(caracteristica_genero('Neo'))

'male'

In [ ]:
classifier.classify(caracteristica_genero('Trinity'))

'female'

Código original de la página [Learning to Classify Text](https://www.nltk.org/book_1ed/ch06.html)

In [ ]:
from nltk.corpus import names
import random
names = ([(name, 'male') for name in names.words('male.txt')] +
          [(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(names)

def gender_features(word):
  return {'last_letter': word[-1]}
gender_features('Shrek')



 	
featuresets = [(gender_features(n), g) for (n,g) in names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.classify(gender_features('Albert'))

'male'

In [ ]:
classifier.classify(gender_features('Jane'))

'female'

In [ ]:
classifier.classify(gender_features('Carmen'))

'male'

Después de entrenar al clasificador se evalúan los resultados. En este ejemplo, el clasificador tuvo una exactitud del 73.8%. Nota que este proceso de evaluación se hace sobre el conjunto de pruebas. En la figura 5 también se pueden observar cuáles fueron las cinco características más importantes para clasificar los nombres. La lista desplegada muestra que los nombres del conjunto de entrenamiento que terminan en a aparecieron 34 veces más en la categoría femenino que masculino. De forma similar, los nombres que terminan en k aparecieron 32.0
 veces más en la categoría femenino que en masculino.

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

0.738


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     38.1 : 1.0
             last_letter = 'k'              male : female =     32.0 : 1.0
             last_letter = 'f'              male : female =     26.8 : 1.0
             last_letter = 'p'              male : female =     11.3 : 1.0
             last_letter = 'v'              male : female =     10.6 : 1.0


El ejemplo siguiente utilizará un clasificador bayesiano para adivinar la categoría de una crítica de películas. Se utilizará el corpus movie_reviews de NLTK.

En la figura 6 se muestra el procedimiento para descargar el corpus, así como una crítica, cuya clasificación es negativa (nota la palabra “neg” al final del registro).

In [ ]:
documentos = [(list(movie_reviews.words(fileid)), category)
         for category in movie_reviews.categories()
         for fileid in movie_reviews.fileids(category)]
random.shuffle(documentos)
print(documentos[5])

(['notting', 'hill', "'", 's', 'trailer', 'is', 'awful', ':', 'a', 'laughless', ',', 'schmaltzy', 'montage', '.', 'the', 'movie', 'was', 'desperately', 'marketed', 'to', 'the', 'anti', '-', 'phantom', 'menace', 'crowd', ',', 'the', 'same', 'lovelorn', 'females', 'who', 'ignored', 'the', 'love', 'letter', '.', 'and', 'it', 'apparently', 'worked', '.', 'perhaps', 'the', 'presence', 'of', 'julia', 'roberts', '-', 'whose', 'allure', 'beyond', 'those', 'perfect', 'teeth', 'still', 'escapes', 'me', '-', 'had', 'a', 'lot', 'to', 'do', 'with', 'notting', 'hill', "'", 's', 'opening', 'weekend', 'success', ',', 'but', 'the', 'film', "'", 's', 'staying', 'power', 'is', 'based', 'on', 'word', 'of', 'mouth', '.', 'allow', 'me', 'to', 'spread', 'some', 'more', 'good', 'buzz', 'for', 'notting', 'hill', '.', 'grant', 'stars', 'as', 'william', 'thacker', ',', 'a', 'travel', '-', 'bookstore', 'owner', 'who', 'works', 'and', 'resides', 'in', 'a', 'tiny', 'english', 'district', 'called', 'notting', 'hill'

El procedimiento de extracción de características se muestra a continuación.

In [ ]:
palabras = nltk.FreqDist(w.lower() for w in movie_reviews.words())
caract_palabras = list(palabras.keys())[:2000]

In [ ]:
def caract_revision(document):
  document_words = set (document)
  features = {}
  for word in caract_palabras:
    features['contiene(%s)' % word] = (word in document_words)
  return features

In [ ]:
print(caract_revision(movie_reviews.words('pos/cv957_8737.txt')))

{'contiene(plot)': True, 'contiene(:)': True, 'contiene(two)': True, 'contiene(teen)': False, 'contiene(couples)': False, 'contiene(go)': False, 'contiene(to)': True, 'contiene(a)': True, 'contiene(church)': False, 'contiene(party)': False, 'contiene(,)': True, 'contiene(drink)': False, 'contiene(and)': True, 'contiene(then)': True, 'contiene(drive)': False, 'contiene(.)': True, 'contiene(they)': True, 'contiene(get)': True, 'contiene(into)': True, 'contiene(an)': True, 'contiene(accident)': False, 'contiene(one)': True, 'contiene(of)': True, 'contiene(the)': True, 'contiene(guys)': False, 'contiene(dies)': False, 'contiene(but)': True, 'contiene(his)': True, 'contiene(girlfriend)': True, 'contiene(continues)': False, 'contiene(see)': False, 'contiene(him)': True, 'contiene(in)': True, 'contiene(her)': False, 'contiene(life)': False, 'contiene(has)': True, 'contiene(nightmares)': False, 'contiene(what)': True, "contiene(')": True, 'contiene(s)': True, 'contiene(deal)': False, 'contiene

El resultado del entrenamiento arroja que el clasificador tiene una exactitud del 84% y de las características más relevantes se puede notar que aquellas críticas que contienen la palabra “unimaginative” se asociaron 8.4 veces más a una crítica negativa que a una positiva en el conjunto de entrenamiento, tal como se muestra a continuación.

In [ ]:
featuresets = [(caract_revision(d),c) for (d,c) in documentos]
train_set, test_set = featuresets[100:],featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

0.8


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
 contiene(unimaginative) = True              neg : pos    =      8.3 : 1.0
    contiene(schumacher) = True              neg : pos    =      7.4 : 1.0
          contiene(mena) = True              neg : pos    =      7.0 : 1.0
        contiene(shoddy) = True              neg : pos    =      7.0 : 1.0
        contiene(suvari) = True              neg : pos    =      7.0 : 1.0


Asegúrate de:

Realizar ejercicios prácticos que incluyan el proceso de clasificación de textos.

Comprender la diferencia entre conjunto de entrenamiento y de pruebas.

Diferenciar los pasos del proceso de clasificación.


# Lecturas

Collins, M. (s.f.). The Naive Bayes Model, Maximum-Likelihood Estimation, and the EM Algorithm. Recuperado de https://www.cs.cornell.edu/courses/cs5740/2016sp/resources/collins_em.pdf

Eisenstein, J. (2018). Natural Language Processing. Recuperado de https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf

Scikit-learn.org. (2020). User Guide. Recuperado de https://scikit-learn.org/stable/user_guide.html

Yoon, S., Byun, S., y Jung, K. (2018). Multimodal Speech Emotion Recognition Using Audio and Text. Recuperado de https://arxiv.org/abs/1810.04635

# Recursos adicionales

edureka! (2020, 30 de julio). What is Text Classification in NLP | NLP Tutorial for Beginners | Edureka | NLP Live – 4 [Archivo de video]. Recuperado de https://www.youtube.com/watch?v=N3Ep7ndcLNE

Simplilearn. (2020, 5 de octubre). Text Classification Using Naive Bayes | Naive Bayes Algorithm In Machine Learning | Simplilearn [Archivo de video]. Recuperado de https://www.youtube.com/watch?v=60pqgfT5tZM